In [1]:
import os
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import branca.colormap as cm
from matplotlib import colors

In [2]:
df_refugee = pd.read_csv("time_series.csv",low_memory=False)
display(df_refugee.size)
df_refugee['Value'] = pd.to_numeric(df_refugee['Value'], errors='coerce')
df_refugee= df_refugee[df_refugee['Value'] > 0]
display(df_refugee.size)

1492205

825275

In [3]:
df_geo  = pd.read_csv("geo.csv")
display(df_geo.head())

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [4]:
df_merge = df_refugee.merge(df_geo, left_on='Country / territory of asylum/residence', right_on='name', how='inner')
df_merge = df_merge[['Year','latitude','longitude','Value','Origin']]
df_merge = df_merge.rename(columns={'latitude': 'latitude_asylum','longitude': 'longitude_asylum'})
display(df_merge.head())

,Year,latitude_asylum,longitude_asylum,Value,Origin
0,1951,-25.274398,133.775136,180000.0,Various/Unknown
1,1965,-25.274398,133.775136,40000.0,Various/Unknown
2,1966,-25.274398,133.775136,40000.0,Various/Unknown
3,1967,-25.274398,133.775136,44000.0,Various/Unknown
4,1968,-25.274398,133.775136,43000.0,Various/Unknown


In [5]:
df_merge = df_merge.merge(df_geo, left_on='Origin', right_on='name', how='inner')
df_merge = df_merge[['Year','latitude','longitude','latitude_asylum','longitude_asylum','Value']]
df_merge = df_merge.rename(columns={'latitude': 'latitude_origin','longitude': 'longitude_origin'})
display(df_merge.head())
df_merge = df_merge.sort_values("Year")

,Year,latitude_origin,longitude_origin,latitude_asylum,longitude_asylum,Value
0,1993,-35.675147,-71.542969,-25.274398,133.775136,89.0
1,1994,-35.675147,-71.542969,-25.274398,133.775136,58.0
2,1995,-35.675147,-71.542969,-25.274398,133.775136,42.0
3,1996,-35.675147,-71.542969,-25.274398,133.775136,6.0
4,1997,-35.675147,-71.542969,-25.274398,133.775136,7.0


In [6]:
df_merge['Value'] = pd.to_numeric(df_merge['Value'], errors='coerce')
df_merge= df_merge[df_merge['Value'] > 0]
display(df_merge)

,Year,latitude_origin,longitude_origin,latitude_asylum,longitude_asylum,Value
70400,1961,-1.940278,29.873888,1.373333,32.290275,52000.0
70167,1961,-1.940278,29.873888,-3.373056,29.918886,30000.0
35492,1961,7.946527,-1.023194,8.619543,0.824782,5000.0
70329,1961,-1.940278,29.873888,-6.369028,34.888822,12000.0
70401,1962,-1.940278,29.873888,1.373333,32.290275,52000.0
70168,1962,-1.940278,29.873888,-3.373056,29.918886,30000.0
70330,1962,-1.940278,29.873888,-6.369028,34.888822,12000.0
35493,1962,7.946527,-1.023194,8.619543,0.824782,5000.0
123324,1963,11.803749,-15.180413,14.497401,-14.452362,6000.0
70169,1963,-1.940278,29.873888,-3.373056,29.918886,34000.0


In [7]:
df_merge.to_csv("final_data.csv", sep=',', encoding='utf-8',index = False)